In [1]:
import os 
os.chdir("C:\\Users\\Pieter-Jan\\Documents\\Work\\Candriam\\nlp\\ESG\\topic_sentiment_analysis")

# Libraries

In [2]:
# data manipulations
import pandas as pd
from sklearn.feature_extraction import text

# data manipulations
import pandas as pd
import numpy as np
import re
from scipy import sparse

# CMT
from contextualized_topic_models.models.ctm import CTM
from contextualized_topic_models.datasets.dataset import CTMDataset
from contextualized_topic_models.evaluation.measures import CoherenceNPMI, InvertedRBO, CoherenceWordEmbeddings
from contextualized_topic_models.utils.preprocessing import SimplePreprocessing

# lDA
from gensim.corpora.dictionary import Dictionary
from gensim.test.utils import common_texts
from gensim.models import LdaMulticore 
import pyLDAvis.gensim
from gensim import corpora, matutils, models, similarities
import pyLDAvis

# BerTopic
from bertopic import BERTopic

# other
import random

# own modules
import modules.preprocessing as preprocess

sorflow\core\framework\types_pb2.py:177: DeprecationWarning: Call to deprecated create function EnumValueDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.EnumValueDescriptor(
C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\types_pb2.py:181: DeprecationWarning: Call to deprecated create function EnumValueDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.EnumValueDescriptor(
C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\types_pb2.py:185: DeprecationWarning: Call to deprecated create function EnumValueDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.EnumValueD

In [3]:
%reload_ext autoreload
%autoreload 2

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Goal

We are going to compare the performance of three **unsupervised** models for topic modelling on ESG documents.

1. Contextualized Topic Modelling (CTM): https://github.com/MilaNLProc/contextualized-topic-models
2. Latent Dirichlet Allocation (LDA): https://radimrehurek.com/gensim/models/ldamodel.html
3. BERTopic: https://github.com/MaartenGr/BERTopic

# Evaluation measures 

1. **Normalized Point-wise Mutual Information (NPMI) (Lau et al.,
2014)**

It measures how much the top-10 words of a topic are related to each other, considering the empirical frequency of the words computed on the
original corpus. τ is a symbolic metric and relies on co-occurrence.

2. **External Word Embeddings Topic Coherence**

As Ding et al. (2018) pointed out, though, topic
coherence computed on the same data is inherently
limited. Coherence computed on an external corpus, on the other hand, correlates much more to
human judgment, but it may be expensive to estimate. Thus, our second metric is an external
word embeddings topic coherence metric, which we compute by adopting a strategy similar to that
described in Ding et al. (2018). First, we compute
the average pairwise cosine similarity of the word
embeddings of the top-10 words in a topic using (Mikolov et al., 2013) embeddings. Then, we
compute the overall average of those values for all
the topics (α).

3. **rank-
biased overlap (RBO) (Webber et al., 2010)**

To evaluate how diverse the topics
generated by a single model are, we use the rank-
biased overlap (RBO) (Webber et al., 2010). RBO
compares two topics of the same model. The key
qualities of this measure are twofold: it allows
disjointedness between the lists of topics (i.e., two
topics can have different words in them) and it is
weighted on the ranking (i.e., two lists that share
some of the same words, albeit at different rankings,
are penalized less than two lists that share the same
words at the highest ranks). We deﬁne ρ as the rank-
biased overlap diversity, that we interpret as the
reciprocal of the standard RBO. ρ is 0 for identical
topics and 1 for completely different topics. Both
metrics are computed on the top-k ranked lists.
Following the state-of-the-art, we consider k = 10.

In [4]:
# global variable used throughout te notebook to update preprocessing steps
UPDATE = False
DIR_REPORTS = 'C:/Users/Pieter-Jan/Documents/Work/Candriam/nlp/ESG/reports/Sectors/'

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Download reports and perform some text processing

In [5]:
df = pd.read_excel("data//Industry_CSR_Datasets.xlsx").dropna().reset_index(drop=True)
df.head()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\Pieter-Jan\Anaconda3\envs\marketDb\lib\site-packages\xlrd\xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\Users\Pieter-Jan\Anaconda3\envs\marketDb\lib\site-packages\xlrd\xlsx.py:312: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\Users\Pieter-Jan\Anaconda3\envs\marketDb\lib\site-packag

,company,industry,sector,year,url
0,Brookfield Property Partners LP,Real Estate Development,Financial,2019.0,https://www.brookfield.com/sites/default/files...
1,PrairieSky Royalty Ltd,Real Estate Development,Financial,2017.0,https://www.prairiesky.com/wp-content/uploads/...
2,PrairieSky Royalty Ltd,Real Estate Development,Financial,2018.0,https://www.prairiesky.com/wp-content/uploads/...
3,PrairieSky Royalty Ltd,Real Estate Development,Financial,2019.0,https://www.prairiesky.com/wp-content/uploads/...
4,Summit Hotel Properties Inc.,Real Estate Development,Financial,2018.0,https://www.responsibilityreports.com/HostedDa...


In [6]:
# download reports and store
df_download, could_not_download = preprocess.download_reports(df=df, directory=DIR_REPORTS, update=True)

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 168/168 [05:32<00:00,  1.98s/it]


In [7]:
df_download.shape

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(168, 6)

In [8]:
# onlyfiles = [f for f in os.listdir(DIR_REPORTS) if os.path.isfile(os.path.join(DIR_REPORTS, f))]
# df = df[~df['company'].isin(could_not_download)]
# df["filename"] = onlyfiles

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
df_processed = preprocess.load_processed_text(
    df_download,
    dir_read_pdf=DIR_REPORTS, 
    columns_to_keep = ["company","industry", "sector", "year", "url", "filename"],
    file_processed_text="output/CRS_processed_pdfMiner.txt",
    n_min_word_paragraph=50, 
    n_max_word_paragraph=125,  
    update=True,
    method_extract_content = "pdfMiner"
    )

INFO    pdfminer.pdfinterp: Processing xobj: <PDFStream(121): raw=28316, {'BitsPerComponent': 8, 'ColorSpace': <PDFObjRef:200>, 'Filter': /'DCTDecode', 'Height': 343, 'Intent': /'RelativeColorimetric', 'Length': 28314, 'Metadata': <PDFObjRef:120>, 'Name': /'X', 'Subtype': /'Image', 'Type': /'XObject', 'Width': 259}>
2020-11-18 23:06:39.424 INFO    pdfminer.pdfpage: Page: {'ArtBox': [0.0, 0.0, 612.0, 792.0], 'BleedBox': [0.0, 0.0, 612.0, 792.0], 'Contents': <PDFObjRef:123>, 'CropBox': [0.0, 0.0, 612.0, 792.0], 'MediaBox': [0.0, 0.0, 612.0, 792.0], 'Parent': <PDFObjRef:192>, 'PieceInfo': {'InDesign': {'DocumentID': b'\xfe\xff\x00x\x00m\x00p\x00.\x00d\x00i\x00d\x00:\x006\x009\x000\x00c\x00a\x00d\x002\x004\x00-\x001\x00e\x00d\x002\x00-\x006\x00b\x004\x00f\x00-\x00a\x005\x008\x00d\x00-\x003\x00e\x004\x001\x00f\x000\x00c\x004\x00f\x004\x00d\x001', 'LastModified': b'\xfe\xff\x00D\x00:\x002\x000\x002\x000\x000\x005\x002\x007\x001\x007\x000\x002\x002\x008\x00Z', 'NumberofPages': 1, 'OriginalDoc

In [10]:
print(df_processed.shape)
df_processed.head()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
(20532, 7)


,company,industry,sector,year,url,filename,paragraph
0,Brookfield Property Partners LP,Real Estate Development,Financial,2019,https://www.brookfield.com/sites/default/files...,BrookfieldPropertyPartnersLP-2019.pdf,ESG Report ESG REPORT Contents OUR EOPLE Human...
1,Brookfield Property Partners LP,Real Estate Development,Financial,2019,https://www.brookfield.com/sites/default/files...,BrookfieldPropertyPartnersLP-2019.pdf,ESG Integration into Our Investment Process Sy...
2,Brookfield Property Partners LP,Real Estate Development,Financial,2019,https://www.brookfield.com/sites/default/files...,BrookfieldPropertyPartnersLP-2019.pdf,private wealth investors For more information ...
3,Brookfield Property Partners LP,Real Estate Development,Financial,2019,https://www.brookfield.com/sites/default/files...,BrookfieldPropertyPartnersLP-2019.pdf,ESTAT INF RASTRUCTURE RENEWABLE POWER PRIVATE ...
4,Brookfield Property Partners LP,Real Estate Development,Financial,2019,https://www.brookfield.com/sites/default/files...,BrookfieldPropertyPartnersLP-2019.pdf,Today our capacity is approximately MW of elec...


In [11]:
# save only the paragraph to a text file
df_processed["paragraph"].to_csv('output/CSR_processed_raw_pdfMiner.txt', sep='\t', index=False, header=False)

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# filter out company names
own_stop_words = ["prairiesky", "cousin", "use", "uk", "france", "landlord", "abs", "london"]
fsi_stop_words = df_processed["company"].unique().tolist() 

# our list contains all english stop words + companies names + specific keywords
stop_words = text.ENGLISH_STOP_WORDS.union(fsi_stop_words)

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
# load spacy model to lematize text
nlp = preprocess.load_spacy_model()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [14]:
# lematize text and remove stopwords
# use method 1

df_processed = preprocess.load_lemmatize(
    data=df_processed, 
    dir_file='output/CSR_processed_cleaned_pdfMiner.txt', 
    stop_words=stop_words, 
    nlp=nlp, 
    method=1, 
    update=True
)

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 20532/20532 [04:58<00:00, 68.68it/s]


## 1. Contextualized Topic Modelling

## 1.1 Prepare data for model

In [15]:
with open('output/CSR_processed_cleaned_pdfMiner.txt',"r") as fr:
    text_cleaned = [doc.split() for doc in fr.read().splitlines()] # load text for NPMI

dictionary = Dictionary(text_cleaned)

'''
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''

dictionary.filter_extremes(no_below=10, no_above=0.10, keep_n= 100000)
print(len(dictionary.token2id))

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-11-18 23:12:42.102 INFO    gensim.corpora.dictionary: adding document #0 to Dictionary(0 unique tokens: [])
2020-11-18 23:12:43.124 INFO    gensim.corpora.dictionary: adding document #10000 to Dictionary(11876 unique tokens: ['asset', 'association', 'audit', 'board', 'brookfield']...)
2020-11-18 23:12:43.977 INFO    gensim.corpora.dictionary: adding document #20000 to Dictionary(18772 unique tokens: ['asset', 'association', 'audit', 'board', 'brookfield']...)
2020-11-18 23:12:44.024 INFO    gensim.corpora.dictionary: built Dictionary(19085 unique tokens: ['asset', 'association', 'audit', 'board', 'brookfiel

In [16]:
bow_corpus = [dictionary.doc2bow(doc) for doc in text_cleaned]

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
tf_array = matutils.corpus2dense(bow_corpus, num_terms=len(dictionary.token2id)).T
tf_array.shape

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(20532, 5016)

In [18]:
# convert to sparse matrix
tf_array_sparse = sparse.csr_matrix(tf_array)
tf_array_sparse

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<20532x5016 sparse matrix of type '<class 'numpy.float32'>'
	with 674538 stored elements in Compressed Sparse Row format>

In [19]:
# create or load bert embeddings (either use raw text or clean text)
# we can expirment with both
embeddings_bert = preprocess.load_bert_embeddings(
        text_dir="output/CSR_processed_raw_pdfMiner.txt", 
        model="distiluse-base-multilingual-cased",
        dir_embeddings="output/CRS_bertEmbeddings_pdfMiner.npy",
        update=True
)

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-11-18 23:12:54.686 INFO    root: Load pretrained SentenceTransformer: distiluse-base-multilingual-cased
2020-11-18 23:12:54.689 INFO    root: Did not find folder distiluse-base-multilingual-cased. Assume to download model from server.
2020-11-18 23:12:54.724 INFO    root: Load SentenceTransformer from folder: C:\Users\Pieter-Jan/.cache\torch\sentence_transformers\sbert.net_models_distiluse-base-multilingual-cased
2020-11-18 23:12:59.160 INFO    root: Use pytorch device: cpu
Batches: 100%|██████████| 103/103 [1:34:15<00:00, 54.91s/it]


In [20]:
embeddings_bert.shape

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(20532, 512)

In [21]:
# ivert dictionary
inv_token2id = {v: k for k, v in dictionary.token2id.items()}
# create dataset
training_dataset = CTMDataset(tf_array_sparse, embeddings_bert, inv_token2id)

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 1.2 Train model

In [22]:
random.seed(69)
ctm = CTM(
    input_size=len(dictionary.token2id), 
    bert_input_size=512, 
    n_components=20, 
    inference_type="combined", 
    num_epochs=10,
    reduce_on_plateau=True
    )
ctm.fit(training_dataset) # run model

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Settings: 
                   N Components: 20
                   Topic Prior Mean: 0.0
                   Topic Prior Variance: 0.95
                   Model Type: prodLDA
                   Hidden Sizes: (100, 100)
                   Activation: softplus
                   Dropout: 0.2
                   Learn Priors: True
                   Learning Rate: 0.002
                   Momentum: 0.99
                   Reduce On Plateau: True
                   Save Dir: None
Epoch: [1/10]	Samples: [20532/205320]	Train Loss: 360.49390376150643	Time: 0:01:07.800099
Epoch: [2/10]	Samples: [41064/205320]	Train Loss: 3

## 1.3 Evaluate topics

In [23]:
cmt_topics_l = ctm.get_topic_lists(10)
cmt_topics_d = {}
for i in range(len(cmt_topics_l)):
    cmt_topics_d[i] = cmt_topics_l[i]

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [46]:
(pd.DataFrame.from_dict(cmt_topics_d).T).to_csv("output/topics_CRS_CMT.csv")
pd.DataFrame.from_dict(cmt_topics_d).T

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6,7,8,9
0,benefit,associate,life,offer,time,pay,wellness,culture,diversity,medical
1,risks,curtailment,cemp,regime,recur,accrue,alternate,competitiveness,favorable,achievable
2,health,safety,survey,training,meeting,construction,supplier,investor,security,issue
3,scope,use,metric,factor,greenhouse,source,indirect,direct,calculate,control
4,garden,plastic,yellow,summer,host,participant,just,preparedness,charity,say
5,landlord,like,electricity,tonne,coverage,intensity,meter,table,epra,obtain
6,leed,certification,street,square,office,star,certify,gold,foot,francisco
7,real,estate,green,global,award,industry,index,leader,association,prologis
8,saving,lighting,solar,instal,system,save,efficient,installation,efficiency,store
9,assurance,review,information,statement,verification,basis,process,level,limited,conclusion


### 1.3.1 Normalized Point-wise Mutual Information

In [25]:
npmi = CoherenceNPMI(texts=text_cleaned, topics=ctm.get_topic_lists(10))
npmi.score()

oherence.text_analysis: 189 batches submitted to accumulate stats from 12096 documents (546473 virtual)
2020-11-19 01:03:16.159 INFO    gensim.topic_coherence.text_analysis: 190 batches submitted to accumulate stats from 12160 documents (549800 virtual)
2020-11-19 01:03:16.271 INFO    gensim.topic_coherence.text_analysis: 191 batches submitted to accumulate stats from 12224 documents (552668 virtual)
2020-11-19 01:03:16.301 INFO    gensim.topic_coherence.text_analysis: 192 batches submitted to accumulate stats from 12288 documents (555693 virtual)
2020-11-19 01:03:16.347 INFO    gensim.topic_coherence.text_analysis: 193 batches submitted to accumulate stats from 12352 documents (559313 virtual)
2020-11-19 01:03:16.436 INFO    gensim.topic_coherence.text_analysis: 194 batches submitted to accumulate stats from 12416 documents (562405 virtual)
2020-11-19 01:03:16.503 INFO    gensim.topic_coherence.text_analysis: 195 batches submitted to accumulate stats from 12480 documents (568909 virtu

0.10005078526162889

### 1.3.2 External Word Embeddings Topic Coherence

In [26]:
CoherenceWordEmbeddings(ctm.get_topic_lists(10)).score()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-11-19 01:03:23.915 INFO    gensim.models.utils_any2vec: loading projection weights from C:\Users\Pieter-Jan/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz
2020-11-19 01:04:25.497 INFO    gensim.models.utils_any2vec: loaded (3000000, 300) matrix from C:\Users\Pieter-Jan/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz


0.14553869

### 1.3.3 Rank-Biased Overlap 

In [27]:
InvertedRBO(ctm.get_topic_lists(10)).score()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.9981349341284962

## 2. Latent Dirichlet Allocation

## 2.1 Train model

In [28]:
num_topics = 20
lda_model =  LdaMulticore(
    corpus=bow_corpus, 
    num_topics = num_topics, 
    id2word = dictionary,                                    
    passes = 10,
    workers = 2
    )

 #14 (0.050): 0.035*"hall" + 0.034*"office" + 0.033*"charter" + 0.030*"square" + 0.021*"foot" + 0.020*"street" + 0.016*"fund" + 0.014*"june" + 0.014*"july" + 0.012*"retail"
2020-11-19 01:06:38.447 INFO    gensim.models.ldamodel: topic diff=0.184392, rho=0.227826
2020-11-19 01:06:38.449 INFO    gensim.models.ldamulticore: PROGRESS: pass 8, dispatched chunk #7 = documents up to #16000/20532, outstanding queue size 6
2020-11-19 01:06:40.352 INFO    gensim.models.ldamulticore: PROGRESS: pass 8, dispatched chunk #8 = documents up to #18000/20532, outstanding queue size 6
2020-11-19 01:06:40.764 INFO    gensim.models.ldamodel: merging changes from 4000 documents into a model of 20532 documents
2020-11-19 01:06:40.777 INFO    gensim.models.ldamodel: topic #0 (0.050): 0.037*"electricity" + 0.022*"natural" + 0.021*"use" + 0.020*"fuel" + 0.017*"factor" + 0.017*"source" + 0.017*"renewable" + 0.015*"travel" + 0.014*"office" + 0.014*"landlord"
2020-11-19 01:06:40.780 INFO    gensim.models.ldamodel:

## 2.2 Evaluate topics

In [29]:
lda_topics_d = {}
lda_topics_l = []
for i in range(num_topics):
    t = [w[0] for w in lda_model.show_topic(i)[0:10]]
    lda_topics_d[i+1] = t
    lda_topics_l.append(t)    

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
# show topics
# evert row is a topic
(pd.DataFrame.from_dict(lda_topics_d).T).to_csv("output/topics_CRS_LDA.csv")
pd.DataFrame.from_dict(lda_topics_d).T

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6,7,8,9
1,electricity,use,natural,fuel,source,travel,factor,renewable,greenhouse,office
2,regency,million,investment,share,market,center,income,lease,shareholder,venture
3,centre,hammerson,retail,shopping,number,park,france,female,positive,male
4,goal,reduction,achieve,efficiency,target,progress,sustainable,commitment,greenhouse,track
5,scope,like,intensity,metric,reduction,control,mtco,electricity,landlord,coverage
6,center,simon,wellness,shopping,mall,tree,station,healthy,health,say
7,outlet,cost,carey,increase,centers,value,good,factory,people,do
8,page,approach,disclosure,organization,material,reporting,content,topic,aspect,indicator
9,issue,supplier,design,plan,local,target,develop,strategy,review,engage
10,training,health,safety,associate,manager,time,leadership,professional,education,hour


### 2.2.1 Normalized Point-wise Mutual Information

In [31]:
npmi = CoherenceNPMI(texts=text_cleaned, topics=lda_topics_l)
npmi.score()

oherence.text_analysis: 189 batches submitted to accumulate stats from 12096 documents (546687 virtual)
2020-11-19 01:07:16.870 INFO    gensim.topic_coherence.text_analysis: 190 batches submitted to accumulate stats from 12160 documents (550014 virtual)
2020-11-19 01:07:16.915 INFO    gensim.topic_coherence.text_analysis: 191 batches submitted to accumulate stats from 12224 documents (552882 virtual)
2020-11-19 01:07:16.992 INFO    gensim.topic_coherence.text_analysis: 192 batches submitted to accumulate stats from 12288 documents (555907 virtual)
2020-11-19 01:07:17.015 INFO    gensim.topic_coherence.text_analysis: 193 batches submitted to accumulate stats from 12352 documents (559527 virtual)
2020-11-19 01:07:17.046 INFO    gensim.topic_coherence.text_analysis: 194 batches submitted to accumulate stats from 12416 documents (562619 virtual)
2020-11-19 01:07:17.135 INFO    gensim.topic_coherence.text_analysis: 195 batches submitted to accumulate stats from 12480 documents (569123 virtu

0.09224300680531514

### 2.2.2 External Word Embeddings Topic Coherence

In [32]:
CoherenceWordEmbeddings(lda_topics_l).score()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-11-19 01:07:26.279 INFO    gensim.models.utils_any2vec: loading projection weights from C:\Users\Pieter-Jan/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz
2020-11-19 01:08:19.617 INFO    gensim.models.utils_any2vec: loaded (3000000, 300) matrix from C:\Users\Pieter-Jan/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz


0.14907268

### 2.2.3 Rank-Biased Overlap 

In [33]:
InvertedRBO(lda_topics_l).score()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.9909485593956391

## 3. Bert Topic

## 3.1 Train model

In [34]:
random.seed(69)

with open("output/CSR_processed_cleaned_pdfMiner.txt","r") as fr:
    docs = [doc for doc in fr.read().splitlines()] 

model = BERTopic(verbose=True)
topics = model.fit_transform(docs, embeddings_bert)

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-11-19 01:09:13,354 - BERTopic - Reduced dimensionality with UMAP
2020-11-19 01:09:13.354 INFO    BERTopic: Reduced dimensionality with UMAP
C:\Users\Pieter-Jan\Anaconda3\envs\marketDb\lib\site-packages\joblib\parallel.py:324: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  warnings.warn('check_pickle is deprecated in joblib 0.12 and will be'
C:\Users\Pieter-Jan\Anaconda3\envs\marketDb\lib\site-packages\joblib\parallel.py:324: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  warnings.warn('check_pickle is deprecated in job

## 3.2 Evaluate topics

In [35]:
topcis_b = model.get_topics()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
# extract words for each topic
topics_k = {}
for k,v in topcis_b.items():
    t_words = []
    for w in v:
        t_words.append(w[0])
    # append the first 10 words
    topics_k[k] = t_words[0:10]

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [43]:
topics_bert = list(topics_k.values())
(pd.DataFrame.from_dict(topics_k).T).to_csv("output/topics_CRS_TBERT.csv")
pd.DataFrame.from_dict(topics_k).T

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6,7,8,9
-1,property,environmental,community,water,building,consumption,energy,sustainability,performance,corporate
0,important,relevance,material,external,organization,supplier,resident,advancement,internally,outside
1,rexford,infill,reposition,renovation,value,truck,benefit,industrial,million,california
2,dialogue,stakeholder,kimco,ongoing,engagement,kimcos,engage,interaction,issue,topic
3,alexandria,alexandrias,science,life,equities,profit,critical,agtech,research,campus
...,...,...,...,...,...,...,...,...,...,...
80,climate,risk,change,relate,scenario,tcfd,weather,physical,financial,extreme
81,risk,carbon,climate,reduction,increase,pricing,cost,strategy,return,relate
82,investor,meeting,conference,survey,quarterly,broker,engagement,feedback,customer,call
83,strategy,sustainability,hammerson,business,clear,positive,mirvac,stakeholder,vision,review


### 3.2.1 Normalized Point-wise Mutual Information

In [38]:
npmi = CoherenceNPMI(texts=text_cleaned, topics=topics_bert)
npmi.score()

oherence.text_analysis: 190 batches submitted to accumulate stats from 12160 documents (547104 virtual)
2020-11-19 01:10:10.523 INFO    gensim.topic_coherence.text_analysis: 191 batches submitted to accumulate stats from 12224 documents (550321 virtual)
2020-11-19 01:10:10.800 INFO    gensim.topic_coherence.text_analysis: 192 batches submitted to accumulate stats from 12288 documents (553430 virtual)
2020-11-19 01:10:10.930 INFO    gensim.topic_coherence.text_analysis: 193 batches submitted to accumulate stats from 12352 documents (556217 virtual)
2020-11-19 01:10:10.978 INFO    gensim.topic_coherence.text_analysis: 194 batches submitted to accumulate stats from 12416 documents (559960 virtual)
2020-11-19 01:10:11.131 INFO    gensim.topic_coherence.text_analysis: 195 batches submitted to accumulate stats from 12480 documents (563025 virtual)
2020-11-19 01:10:11.303 INFO    gensim.topic_coherence.text_analysis: 196 batches submitted to accumulate stats from 12544 documents (569556 virtu

0.17185044220753978

### 3.2.2 External Word Embeddings Topic Coherence

In [39]:
CoherenceWordEmbeddings(topics_bert).score()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-11-19 01:10:29.902 INFO    gensim.models.utils_any2vec: loading projection weights from C:\Users\Pieter-Jan/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz
2020-11-19 01:11:25.882 INFO    gensim.models.utils_any2vec: loaded (3000000, 300) matrix from C:\Users\Pieter-Jan/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz


0.15327418

### 3.2.3 Rank-Biased Overlap 

In [40]:
InvertedRBO(topics_bert).score()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.9908176773662869